# Task

Las tres imágenes cargadas en la celda inicial, han sido extraidas de las imágenes de mayor tamaño presentes en la carpeta. La tarea consiste en extraer características (geométricas y/o visuales) e identificar patrones que permitan distinguir las partículas de cada una de las tres clases, evaluando los aciertos y fallos con las imágenes completas considerando las métricas mostradas y la matriz de confusión. La matriz de confusión, muestra para cada clase el número de muestras que se clasifican correctamente de dicha clase, y el número de muestras que se clasifican incorrectamente por cada una de las otras dos clases.

En el trabajo [SMACC: A System for Microplastics Automatic Counting and Classification](https://doi.org/10.1109/ACCESS.2020.2970498), las características geométricas utilizadas fueron:

- Área en píxeles
- Perímetro en píxeles
- Compacidad (relación entre el cuadrado del perímetro y el área de la partícula)
- Relación del área de la partícula con la del contenedor
- Relación del ancho y el alto del contenedor
- Relación entre los ejes de la elipse ajustada
- Definido el centroide, relación entre las distancias menor y mayor al contorno

Si no se quedan satisfechos con la segmentación obtenida, es el mundo real, también en el README comento técnicas recientes de segmentación, que podrían despertar su curiosidad.

In [ ]:
# Import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
def perimeter_area_ratio(contour):
    """
    Computes the compactness of the blob, calculated as the ratio of the square of the perimeter to the area.
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    
    Returns:
    The ratio of the square of the perimeter to the area, which can give a measure of how compact the shape is.
    """
    # Calculate the area of the contour using cv2.contourArea, which computes the area enclosed by the contour.
    area = cv2.contourArea(contour)
    
    # Calculate the perimeter (arc length) of the contour using cv2.arcLength.
    # The second argument 'True' indicates that the contour is closed.
    perimeter = cv2.arcLength(contour, True)
    
    # Return the compactness, which is the ratio of the square of the perimeter to the area.
    # A more compact shape (like a circle) will have a lower value, while irregular shapes will have a higher value.
    return perimeter**2 / area

In [ ]:
def blob_area_and_bounding_box_ratio(contour):
    """
    Computes the ratio between the area of the blob (the shape) and the area of its minimum bounding rectangle.
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    
    Returns:
    The ratio of the blob's area to the area of its minimum bounding box.
    """
    # Calculate the area of the blob (contour) using cv2.contourArea, which gives the area enclosed by the contour.
    blob_area = cv2.contourArea(contour)
    
    # Compute the minimum area bounding rectangle that can enclose the contour using cv2.minAreaRect.
    # The function returns a rotated rectangle. The rect[1] contains the width and height of this rectangle.
    rect = cv2.minAreaRect(contour)
    
    # Extract the width and height of the minimum bounding rectangle.
    width, height = rect[1]
    
    # Calculate the area of the minimum bounding rectangle (width * height).
    rect_area = width * height
    
    # Return the ratio of the blob's area to the rectangle's area.
    # A higher value indicates that the blob occupies most of the bounding box, while a lower value suggests a looser fit.
    return blob_area / rect_area

In [ ]:
def bounding_box_width_and_height_ratio(contour):
    """
    Computes the ratio between the width and height of the bounding box of a blob (shape).
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    
    Returns:
    The ratio of the width to the height of the minimum bounding rectangle of the contour.
    """
    # Compute the minimum area bounding rectangle that encloses the contour using cv2.minAreaRect.
    # The function returns a rotated rectangle. The rect[1] contains the width and height of this rectangle.
    rect = cv2.minAreaRect(contour)
    
    # Extract the width and height of the minimum bounding rectangle.
    width, height = rect[1]
    
    # Return the ratio of the width to the height of the bounding rectangle.
    # This can provide insight into the elongation of the shape; a value near 1 indicates a square-like shape,
    # while higher or lower values indicate elongated shapes.
    return width / height

In [ ]:
def fitted_ellipse_axises_ratio(contour):
    """
    Computes the ratio between the major and minor axes of an ellipse fitted to the contour.
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    
    Returns:
    The ratio of the major axis to the minor axis of the ellipse fitted to the contour.
    Returns 0 if the contour has fewer than 5 points (since at least 5 points are required to fit an ellipse).
    """
    # Check if the contour has fewer than 5 points; fitting an ellipse requires at least 5 points.
    if len(contour) < 5:
        return 0
    
    # Fit an ellipse to the contour using cv2.fitEllipse.
    # The function returns an ellipse defined by its center, axes, and rotation angle.
    ellipse = cv2.fitEllipse(contour)
    
    # Extract the lengths of the major and minor axes from the ellipse.
    # ellipse[1] gives a tuple (axis_1, axis_2), where axis_1 is the major axis and axis_2 is the minor axis.
    axis_1, axis_2 = ellipse[1]
    
    # Return the ratio of the major axis to the minor axis.
    # This ratio provides insight into the elongation of the shape: a value close to 1 indicates a circular shape,
    # while a larger ratio suggests an elongated, elliptical shape.
    return axis_1 / axis_2

In [ ]:
def aspect_ratio(contour):
    """
    Computes the aspect ratio, which is the ratio of the width to the height of the bounding rectangle of the contour.
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    
    Returns:
    The aspect ratio (width/height) of the bounding rectangle that encloses the contour.
    """
    # Compute the bounding rectangle that tightly encloses the contour using cv2.boundingRect.
    # The function returns the top-left corner (x, y) and the width (w) and height (h) of the rectangle.
    x, y, w, h = cv2.boundingRect(contour)
    
    # Calculate the aspect ratio, which is the width divided by the height.
    # This gives a sense of the shape's proportions: a value near 1 indicates a square-like shape,
    # while values greater or less than 1 indicate a more elongated shape.
    aspect_ratio = float(w) / h
    
    # Return the calculated aspect ratio.
    return aspect_ratio

In [ ]:
def extent(contour):
    """
    Computes the extent, which is the ratio of the contour area to the area of its bounding rectangle.
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    
    Returns:
    The extent (contour area / bounding rectangle area), which provides a measure of how much space the contour occupies within its bounding box.
    """
    # Calculate the area of the contour using cv2.contourArea, which gives the area enclosed by the contour.
    area = cv2.contourArea(contour)
    
    # Compute the bounding rectangle that encloses the contour using cv2.boundingRect.
    # The function returns the top-left corner (x, y) and the width (w) and height (h) of the rectangle.
    x, y, w, h = cv2.boundingRect(contour)
    
    # Calculate the area of the bounding rectangle (width * height).
    rect_area = w * h
    
    # Calculate the extent, which is the ratio of the contour's area to the bounding rectangle's area.
    # A higher extent value means the contour occupies a larger portion of the bounding box, while a lower value suggests the contour is more irregular or spread out.
    extent = float(area) / rect_area
    
    # Return the calculated extent.
    return extent

In [ ]:
def solidity(contour):
    """
    Computes the solidity, which is the ratio of the contour area to the area of its convex hull.
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    
    Returns:
    The solidity (contour area / convex hull area), which measures how 'solid' or 'convex' a shape is.
    """
    # Calculate the area of the contour using cv2.contourArea, which gives the area enclosed by the contour.
    area = cv2.contourArea(contour)
    
    # Compute the convex hull of the contour using cv2.convexHull.
    # The convex hull is the smallest convex shape that encloses the contour.
    hull = cv2.convexHull(contour)
    
    # Calculate the area of the convex hull using cv2.contourArea.
    hull_area = cv2.contourArea(hull)
    
    # Calculate the solidity, which is the ratio of the contour's area to the convex hull's area.
    # A solidity value close to 1 indicates the shape is nearly convex, while a lower value indicates the shape has more concavities.
    solidity = float(area) / hull_area
    
    # Return the calculated solidity.
    return solidity

In [ ]:
def circularity(contour):
    """
    Computes a measure of circularity, where lower values indicate the shape is closer to a perfect circle.
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    
    Returns:
    A value representing how circular the shape is. The lower the value, the closer the shape is to a circle.
    """
    # Calculate the area of the contour using cv2.contourArea, which gives the area enclosed by the contour.
    area = cv2.contourArea(contour)
    
    # Calculate the perimeter (arc length) of the contour using cv2.arcLength.
    # The second argument 'True' indicates that the contour is closed.
    perimeter = cv2.arcLength(contour, True)
    
    # Compute the ratio of the contour area to its perimeter.
    # This gives an initial measure of circularity; for a perfect circle, this value would be relatively high.
    area_perimeter_ratio = area / perimeter
    
    # Compute the minimum enclosing circle that encloses the contour using cv2.minEnclosingCircle.
    # The function returns the center coordinates (cx, cy) and the radius (r) of the enclosing circle.
    (cx, cy), r = cv2.minEnclosingCircle(contour)
    
    # Calculate the final circularity measure by subtracting half of the radius from the area-perimeter ratio.
    # The closer the result is to zero, the more circular the shape is.
    return abs(area_perimeter_ratio - r / 2)

In [ ]:
def mean_colour(contour, img):
    """
    Computes the average color or intensity within the area defined by the contour.
    
    Parameters:
    contour: A contour of the shape, typically obtained from an image using methods like cv2.findContours.
    img: The input image (in BGR format) from which the mean color/intensity will be calculated.
    
    Returns:
    The average grayscale intensity value within the contour region.
    """
    # Convert the input image to grayscale using cv2.cvtColor.
    # This simplifies the color information to intensity values for easier processing.
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Create an empty mask (same size as the grayscale image) initialized with zeros (black).
    mask = np.zeros(img_grey.shape, np.uint8)
    
    # Draw the contour on the mask. The contour is filled with white (255) inside the shape (-1 indicates filling the contour).
    # This mask highlights the area enclosed by the contour.
    cv2.drawContours(mask, [contour], 0, 255, -1)
    
    # Find the pixel points in the masked area (i.e., the region inside the contour) using np.nonzero.
    pixelpoints = np.transpose(np.nonzero(mask))
    
    # Compute the mean intensity value of the grayscale image within the contour area using cv2.mean.
    # The mask limits the mean calculation to the area inside the contour.
    mean_val = cv2.mean(img_grey, mask=mask)
    
    # Return the mean grayscale intensity value. The first value of mean_val corresponds to the intensity, 
    # since mean_val is a tuple containing the mean of each color channel (in this case, grayscale has only one channel).
    return mean_val[0]

In [ ]:
# List of functions responsible for extracting specific features from a contour
feature_extracting_functions = [
    perimeter_area_ratio,                  # Ratio of perimeter squared to area
    blob_area_and_bounding_box_ratio,      # Ratio of blob area to bounding box area
    bounding_box_width_and_height_ratio,   # Ratio of width to height of the bounding box
    fitted_ellipse_axises_ratio,           # Ratio of major to minor axis of the fitted ellipse
    aspect_ratio,                          # Ratio of width to height of bounding rectangle
    extent,                                # Ratio of contour area to bounding rectangle area
    solidity,                              # Ratio of contour area to convex hull area
    circularity,                           # Custom circularity measure
]

In [ ]:
def get_contour_feature_vector(contour, img):
    """
    Extracts a feature vector for a given contour by applying the list of feature extracting functions.
    
    Parameters:
    contour: The contour from which the features will be extracted.
    img: The image in which the contour is found, used for color/intensity-related features.
    
    Returns:
    A list containing the computed feature values for the contour.
    """
    # Apply each feature extraction function to the contour and collect the results in a list.
    contour_feature_vector = [function(contour=contour) for function in feature_extracting_functions]
    
    # Append the mean color/intensity value from the 'mean_colour' function to the feature vector.
    contour_feature_vector.append(mean_colour(contour, img))
    
    # Return the complete feature vector for the contour.
    return contour_feature_vector

In [ ]:
def feature_extraction(microplastic_external_contours, img):
    """
    Extracts and averages the features for a list of contours from the input image.
    
    Parameters:
    microplastic_external_contours: List of external contours of microplastic particles.
    img: The image from which the features are extracted.
    
    Returns:
    A list containing the averaged feature values for all contours.
    """
    # Initialize a list of empty lists to store feature values.
    # There are len(feature_extracting_functions) + 1 because we include the mean color feature.
    features = [[] for _ in range(len(feature_extracting_functions) + 1)]
    
    # Process each contour to extract its feature vector.
    for contour in microplastic_external_contours:
        # Extract the feature vector for the current contour.
        feature_vector = get_contour_feature_vector(contour, img)
        
        # Append each feature value from the vector to the corresponding feature list.
        for i, feature_value in enumerate(feature_vector):
            features[i].append(feature_value)
    
    # Calculate the average value of each feature across all contours.
    # For each feature list, we sum the values and divide by the number of contours (average).
    averaged_features = [sum(feature_list) / len(feature_list) for feature_list in features]
    
    # Return the averaged feature values.
    return averaged_features

#### Incidencia: Detección de contornos no deseados
Realizando la detección de características me he encontrado con situaciones en las que el área del contorno es nula (división por cero). Algo preocupante puesto que demuestra que se están detectando contornos muy pequeños que no se corresponden con el elemento microplástico, objeto de interés.

Esto se puede visualizar descomentando la llamada a la función escrita a continuación, que muestra el area de los contornos detectados y los pinta en la imagen.

La solución ha sido el establecimiento de un umbral para trabajar solamente con aquellos contornos que superen el umbral.

In [ ]:
def show_all_contours(img, contours):
    """
    Displays the bounding rectangles for all contours on the image and plots their respective areas.
    
    Parameters:
    img: The input image where contours will be drawn.
    contours: A list of contours, typically obtained from an image using methods like cv2.findContours.
    
    Functionality:
    - It draws a bounding rectangle around each contour.
    - It also plots the area of each contour in a separate plot.
    """
    # Calculate the area of each contour using cv2.contourArea and store it in the 'areas' list.
    areas = [cv2.contourArea(contour) for contour in contours]
    
    # For each contour, calculate the bounding rectangle and draw it on the image.
    for contour in contours:
        # Get the x, y coordinates and width (w), height (h) of the bounding rectangle around the contour.
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw the rectangle on the image in green (RGB: (50, 255, 50)) with a thickness of 1.
        img = cv2.rectangle(img, (x, y), (x + w, y + h), (50, 255, 50), 1)
    
    # Create a subplot layout to display two plots side by side.
    plt.subplot(121)
    
    # Label the y-axis as "Area" for the area plot.
    plt.ylabel("Area")
    
    # Plot the areas of the contours. The x-axis represents the contour index, and the y-axis represents the area.
    plt.plot(areas)
    
    # Show the modified image with the bounding rectangles in the second subplot.
    plt.subplot(122)
    
    # Turn off the axis for the image display (no gridlines or labels).
    plt.axis("off")
    
    # Display the image with the drawn contours.
    plt.imshow(img)
    
    # Show both plots (the contour areas and the image with bounding rectangles).
    plt.show()

#### Llamada principal
Se procede a extraer las características de fragmentos, pellets y alquitrán

In [ ]:
# Dictionary containing file names for images to be used for feature extraction.
# The keys represent the type of microplastic, and the values are the file paths to the images.
feature_extraction_image_file_names = {
    "Fragments": "./Images/Waste/FRA.png",
    "Pellets": "./Images/Waste/PEL.png",
    "Tar": "./Images/Waste/TAR.png"
}

# Dictionary to store the extracted features for each type of microplastic.
features = dict()

# Loop over each microplastic type and its corresponding image file.
for microplastic, filename in feature_extraction_image_file_names.items():
    # Load the image from the file using OpenCV's imread function.
    microplastic_img = cv2.imread(filename)

    # Convert the image to grayscale using cv2.cvtColor.
    microplastic_grey = cv2.cvtColor(microplastic_img, cv2.COLOR_BGR2GRAY)

    # Apply Otsu's binarization (thresholding) to convert the grayscale image into a binary image.
    # The binary image is inverted (THRESH_BINARY_INV), and Otsu's method automatically determines the threshold value.
    ret, microplastic_otsu = cv2.threshold(microplastic_grey, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Extract the external contours from the binary image using cv2.findContours.
    # cv2.RETR_EXTERNAL retrieves only the outer contours, and cv2.CHAIN_APPROX_SIMPLE compresses the contour representation.
    microplastic_external_contours, microplastic_external_hierarchy = cv2.findContours(microplastic_otsu, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Uncomment the following line to display all contours with bounding rectangles.
    # show_all_contours(microplastic_img.copy(), microplastic_external_contours)

    # Define area thresholds to filter out very small or very large contours.
    min_area_threshold = 750
    max_area_threshold = 30000
    
    # Lambda function to filter contours based on area.
    # Only contours with an area between the min and max thresholds are kept.
    filter = lambda contour: cv2.contourArea(contour) > min_area_threshold and cv2.contourArea(contour) < max_area_threshold
    
    # Apply the filter to the list of external contours.
    # Only contours that pass the area threshold check are kept.
    microplastic_filtered_external_contours = [contour for contour in microplastic_external_contours if filter(contour=contour)]

    # Extract features from the filtered contours using the feature_extraction function.
    # The extracted features for the current microplastic type are stored in the 'features' dictionary.
    features[microplastic] = feature_extraction(microplastic_filtered_external_contours, microplastic_img)

### Resultados y matriz de confusión

In [ ]:
def key_with_max_value(d):
    """
    Returns the key corresponding to the maximum value in the dictionary.
    
    Parameters:
    d: A dictionary where the values are comparable (e.g., numbers).
    
    Returns:
    The key associated with the highest value in the dictionary.
    """
    # Get the list of values and keys from the dictionary.
    v = list(d.values())
    k = list(d.keys())
    
    # Find the index of the maximum value in the values list, and return the corresponding key from the keys list.
    return k[v.index(max(v))]

In [ ]:
def process_contour(contour, img, reference_features):
    """
    Given a contour, this function determines the type of microplastic and draws the classification confidence on the image.
    
    Parameters:
    contour: The contour of the object to be classified.
    img: The image where the contour is found and where the result will be drawn.
    reference_features: A dictionary containing the reference feature values for different microplastic types.
    
    Returns:
    The type of microplastic that has the highest classification score.
    """
    
    # Extract the feature vector for the given contour using the get_contour_feature_vector function.
    contour_feature_vector = get_contour_feature_vector(contour, img)
    
    # Initialize a list of lists that stores the minimum feature difference and the corresponding microplastic type.
    # Each element starts as [9999, 0] (large initial difference, no microplastic type assigned).
    accumulative_diff_list = [[9999, 0] for _ in range(len(contour_feature_vector))]
    
    # Dictionary to keep track of the number of times each microplastic type is selected based on the smallest feature difference.
    results = dict()

    # Compare the contour's feature vector to the reference feature values for each microplastic type.
    for microplastic_type, reference_feature_values in reference_features.items():
        # Initialize the result counter for this microplastic type.
        results[microplastic_type] = 0
        
        # For each feature in the contour's feature vector, calculate the absolute difference from the reference.
        for i, v in enumerate(contour_feature_vector):
            feature_difference = abs(contour_feature_vector[i] - reference_feature_values[i])
            
            # If the difference is smaller than the current smallest difference for this feature,
            # update the accumulative_diff_list with the new smaller difference and the associated microplastic type.
            if feature_difference < accumulative_diff_list[i][0]:
                accumulative_diff_list[i] = [feature_difference, microplastic_type]

    # Process the results: tally up how many times each microplastic type had the smallest difference for a feature.
    for difference, microplastic_type in accumulative_diff_list:
        results[microplastic_type] += 1

    # Determine the total number of features in the feature vector.
    number_of_features = len(contour_feature_vector)
    
    # Create a string to display the classification results as percentages.
    results_as_percentage = ""
    for microplastic_type, value in results.items():
        # Calculate the percentage of features that correspond to each microplastic type.
        results_as_percentage += f"{microplastic_type[0:3]}: {value * 100 / number_of_features:.1f}%. "
    
    # Draw a bounding rectangle around the contour on the image.
    x, y, w, h = cv2.boundingRect(contour)
    img = cv2.rectangle(img, (x, y), (x + w, y + h), (50, 255, 50), 2)
    
    # Add the classification results as text near the contour on the image.
    cv2.putText(img, results_as_percentage, (x - w, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (50, 255, 50), 2)

    # Return the microplastic type that has the highest count in the 'results' dictionary (the most likely classification).
    return key_with_max_value(results)

En este caso, la matriz de confusión sólo tiene una fila puesto que la clasificación real es siempre la misma para todos los elementos de cada imagen

In [ ]:
# Dictionary containing the file paths of the images for each type of microplastic
contrast_image_file_names = {
    "Fragments": "./Images/Waste/fragment-03-olympus-10-01-2020.JPG",
    "Pellets": "./Images/Waste/pellet-03-olympus-10-01-2020.JPG",
    "Tar": "./Images/Waste/tar-03-olympus-10-01-2020.JPG"
}

# Loop through each type of microplastic and its corresponding image file
for microplastic, filename in contrast_image_file_names.items():
    # Load the image from the file
    microplastic_img = cv2.imread(filename)

    # Convert the image to grayscale for easier contour detection
    microplastic_grey = cv2.cvtColor(microplastic_img, cv2.COLOR_BGR2GRAY)

    # Apply OTSU's thresholding (inverted) to create a binary image for contour detection
    ret, microplastic_otsu = cv2.threshold(microplastic_grey, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Extract external contours from the thresholded image
    # These contours correspond to the shapes of the objects in the image
    microplastic_external_contours, microplastic_external_hierarchy = cv2.findContours(
        microplastic_otsu, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # Create a histogram to count the occurrences of each type of microplastic detected in the image
    histogram = {
        "Fragments": 0,
        "Pellets": 0,
        "Tar": 0
    }

    # Define the area threshold for filtering out irrelevant contours (noise or very large objects)
    min_area_threshold = 750
    max_area_threshold = 30000

    # Filter the contours based on area, keeping only those with an area within the defined threshold
    filter = lambda contour: cv2.contourArea(contour) > min_area_threshold and cv2.contourArea(contour) < max_area_threshold
    microplastic_filtered_external_contours = [
        contour for contour in microplastic_external_contours if filter(contour=contour)
    ]

    # Process each filtered contour to classify it and update the histogram
    for contour in microplastic_filtered_external_contours:
        # Ignore any contour with an area of 0 (invalid contours)
        if cv2.contourArea(contour) <= 0:
            continue
        
        # Classify the contour and update the histogram for the predicted microplastic type
        histogram[process_contour(contour, microplastic_img, features)] += 1

    # Save the image with detected contours and classification results drawn on it
    cv2.imwrite(f"{microplastic}-result-of-detection.jpg", microplastic_img)

    # Create a confusion matrix to evaluate the accuracy of the classification
    # 'true_row' tracks how many contours were classified correctly (True Positives) vs. misclassified (False Positives)
    true_row = [0, 0]  # [True Positives, False Positives]
    for key, value in histogram.items():
        if key == microplastic:
            true_row[0] = value  # Correct classifications for the current microplastic type
        else:
            true_row[1] += value  # Incorrect classifications for the current microplastic type
    
    # Construct the confusion matrix with a row for the current microplastic type and another for errors
    confusion_matrix = [true_row, [0, 0]]  # The second row can store additional confusion data if needed

    # Visualize the confusion matrix using matplotlib
    fig, ax = plt.subplots(figsize=(7.5, 7.5))
    ax.matshow(confusion_matrix, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(len(confusion_matrix)):
        for j in range(len(confusion_matrix[0])):
            ax.text(x=j, y=i, s=confusion_matrix[i][j], va='center', ha='center', size='xx-large')
    
    # Label the confusion matrix plot
    plt.xlabel('Predictions', fontsize=18)
    plt.ylabel('Actual values', fontsize=18)
    plt.title(f"Confusion matrix for {microplastic}", fontsize=18)
    plt.show()

    # Display the image with detected contours and results
    plt.axis("off")
    plt.title(f"{microplastic} image; Results: {histogram}")
    plt.imshow(microplastic_img)
    plt.show()